In [1]:
from glob import glob 
file_locs = "../datacrawl/crawled_data/*/*.json"
import json

def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {file_path} was not found")
    except json.JSONDecodeError as e:
        raise json.JSONDecodeError(f"Error parsing JSON file: {str(e)}", e.doc, e.pos)


In [2]:
data_1 = read_json_file('../datacrawl/crawled_data/61723086/data.json')


In [3]:
glob(file_locs)[0]

'../datacrawl/crawled_data/61723086/data.json'

In [4]:
# data_1.keys()

In [5]:
def get_json_schema(data):
    """
    Extract schema from JSON data
    Returns a dict representing the structure, with types as values
    """
    if isinstance(data, dict):
        return {
            key: get_json_schema(value)
            for key, value in data.items()
        }
    elif isinstance(data, list):
        # If list is empty, return empty list schema
        if not data:
            return ["array (empty)"]
        # If all items are same type, just return first item's schema
        if all(isinstance(x, type(data[0])) for x in data):
            return [get_json_schema(data[0])]
        # If mixed types, return schema for each unique type
        return [get_json_schema(x) for x in data]
    else:
        return type(data).__name__

def print_schema(schema, indent=0):
    """Pretty print the schema"""
    if isinstance(schema, dict):
        result = "{\n"
        for key, value in schema.items():
            result += " " * (indent + 2) + f'"{key}": '
            result += print_schema(value, indent + 2)
            result += ",\n"
        result += " " * indent + "}"
        return result
    elif isinstance(schema, list):
        if len(schema) == 1:
            return f"[{print_schema(schema[0], indent)}]"
        else:
            result = "[\n"
            for item in schema:
                result += " " * (indent + 2)
                result += print_schema(item, indent + 2)
                result += ",\n"
            result += " " * indent + "]"
            return result
    else:
        return f'"{schema}"'



schema = get_json_schema(data_1)
print(print_schema(schema))

{
  "configuration": {
    "actTimeout": "int",
    "env_cfg": {
      "map_height": "int",
      "map_width": "int",
      "match_count_per_episode": "int",
      "max_steps_in_match": "int",
      "max_units": "int",
      "num_teams": "int",
      "unit_move_cost": "int",
      "unit_sap_cost": "int",
      "unit_sap_range": "int",
      "unit_sensor_range": "int",
    },
    "episodeSteps": "int",
    "runTimeout": "int",
    "seed": "int",
  },
  "description": "str",
  "id": "str",
  "info": {
    "EpisodeId": "int",
    "LiveVideoPath": "NoneType",
    "TeamNames": ["str"],
  },
  "name": "str",
  "rewards": ["int"],
  "schema_version": "int",
  "specification": {
    "action": {
      "default": "int",
      "description": "str",
      "type": "str",
    },
    "agents": ["int"],
    "configuration": {
      "actTimeout": {
        "default": "int",
        "description": "str",
        "minimum": "int",
        "type": "str",
      },
      "env_cfg": {
        "description": 

In [6]:
data_1.keys()

dict_keys(['configuration', 'description', 'id', 'info', 'name', 'rewards', 'schema_version', 'specification', 'statuses', 'steps', 'title', 'version'])

In [7]:
obs = data_1["steps"]#[0][0]["observation"]["obs"]

In [8]:
data_1["steps"][15][0]["action"]

[[2, 0, 0],
 [4, 0, 0],
 [3, 0, 0],
 [2, 0, 0],
 [2, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [9]:
# which can be 0 for doing nothing, 1 to move up, 2 to move right, 3 to move down, 4 to move left, and 5 to sap a tile.

In [10]:
# obs.keys()

In [11]:
import torch
def process_observation(replay_observation):
    """Convert replay observation into tensor format"""
    return {
        'map_features': {
            'energy': torch.FloatTensor(replay_observation['map_features']['energy']),
            'tile_type': torch.FloatTensor(replay_observation['map_features']['tile_type'])
        },
        'sensor_mask': torch.FloatTensor(replay_observation['sensor_mask']),
        'units': {
            'position': torch.FloatTensor(replay_observation['units']['position']),
            'energy': torch.FloatTensor(replay_observation['units']['energy'])
        },
        'units_mask': torch.FloatTensor(replay_observation['units_mask']),
        'relic_nodes': torch.FloatTensor(replay_observation['relic_nodes']),
        'energy_nodes': torch.FloatTensor(replay_observation['energy_nodes'])
    }

In [12]:
import torch
import numpy as np

def parse_observation(obs):
    """
    Convert raw observation dict into tensor format required by LuxAIObservationEncoder
    
    Args:
        raw_obs (dict): Raw observation dictionary from the environment
        
    Returns:
        dict: Processed observation with tensors
    """
    
    return {
        'map_features': {
            'energy': torch.FloatTensor(obs['map_features']['energy']).unsqueeze(0),
            'tile_type': torch.FloatTensor(obs['map_features']['tile_type']).unsqueeze(0)
        },
        'sensor_mask': torch.FloatTensor(obs['sensor_mask']).unsqueeze(0),
        'units': {
            'position': torch.FloatTensor(obs['units']['position']).unsqueeze(0),
            'energy': torch.FloatTensor(obs['units']['energy']).unsqueeze(0)
        },
        'units_mask': torch.FloatTensor(obs['units_mask']).unsqueeze(0),
        'relic_nodes': torch.FloatTensor(obs['relic_nodes']).unsqueeze(0),
        'relic_nodes_mask': torch.FloatTensor(obs['relic_nodes_mask']).unsqueeze(0)
    }

In [13]:
np.mean(obs["sensor_mask"] )

TypeError: list indices must be integers or slices, not str

In [14]:
from dataclasses import dataclass
from typing import List, Dict, Any
@dataclass
class GameStats:
    round_won: bool
    relic_points: int
    total_energy: int
    relic_tiles_found: float
    area_explored: float


In [15]:
obs["map_features"]

TypeError: list indices must be integers or slices, not str

In [16]:
# obs['units']['energy']

In [17]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [18]:
action = each_player["action"]

NameError: name 'each_player' is not defined

In [19]:
action

NameError: name 'action' is not defined

In [20]:
import torch
import numpy as np

def env_to_network_actions(env_actions):
    """
    Convert environment actions to network format
    
    Args:
        env_actions: List/array of shape (N, 3) where each action is [action_type, dx, dy]
                    action_type: 0=nothing, 1=up, 2=right, 3=down, 4=left, 5=sap
    
    Returns:
        dict containing:
            actions_taken: Tensor of action indices
            units_selected: Tensor of unit indices for active units
            coords_taken: Tensor of coordinate offsets
    """
    # Convert to numpy if not already
    env_actions = np.array(env_actions)
    
    # Find active units (where action_type != 0)
    active_units = np.where(env_actions[:, 0] != 0)[0]
    
    if len(active_units) == 0:
        # Handle case where no actions are taken
        return {
            'actions_taken': torch.tensor([[0]]),  # No-op action
            'units_selected': torch.tensor([[0]]),  # First unit
            'coords_taken': torch.zeros(1, 2)  # No movement
        }
    
    # For each active unit, convert their action
    action_indices = []
    coord_offsets = []
    
    for unit_idx in active_units:
        action = env_actions[unit_idx]
        action_type = action[0]
        
        if action_type == 5:  # Sap action
            action_idx = 1  # Assuming 1 represents "Zap" in your network
            dx, dy = action[1], action[2]
        else:  # Movement actions
            action_idx = 1  #  0 represents "Move"
            # Convert cardinal directions to dx, dy
            if action_type == 1:    # Up
                dx, dy = 0, -1
            elif action_type == 2:  # Right
                dx, dy = 1, 0
            elif action_type == 3:  # Down
                dx, dy = 0, 1
            elif action_type == 4:  # Left
                dx, dy = -1, 0
                
        action_indices.append(action_idx)
        coord_offsets.append([dx, dy])
    
    # Convert to tensors
    actions_taken = torch.tensor(action_indices).view(-1, 1)
    units_selected = torch.tensor(active_units).view(-1, 1)
    coords_taken = torch.tensor(coord_offsets, dtype=torch.float)
    
    return {
        'actions_taken': actions_taken,
        'units_selected': units_selected,
        'coords_taken': coords_taken
    }

def network_to_env_actions(network_outputs, num_units=16):
    """
    Convert network outputs back to environment action format
    
    Args:
        network_outputs: Dictionary containing network outputs
            - action_probs: Action type probabilities
            - unit_probs: Unit selection probabilities
            - offset_x: X coordinate offsets
            - offset_y: Y coordinate offsets
        num_units: Number of units in the environment
    
    Returns:
        numpy array of shape (N, 3) containing environment actions
    """
    # Initialize empty action array
    env_actions = np.zeros((num_units, 3), dtype=int)
    
    # Get selected actions and units
    actions = torch.argmax(network_outputs['action_probs'], dim=1)
    units = torch.argmax(network_outputs['unit_probs'], dim=1)
    
    # Get coordinate offsets
    offset_x = network_outputs['offset_x'].detach().numpy()
    offset_y = network_outputs['offset_y'].detach().numpy()
    
    for i, (action, unit) in enumerate(zip(actions, units)):
        if action == 0:  # Move action
            # Convert offset to cardinal direction
            dx = offset_x[i]
            dy = offset_y[i]
            
            # Convert to closest cardinal direction
            if abs(dx) > abs(dy):
                if dx > 0:
                    action_type = 2  # Right
                else:
                    action_type = 4  # Left
            else:
                if dy > 0:
                    action_type = 3  # Down
                else:
                    action_type = 1  # Up
                    
            env_actions[unit] = [action_type, 0, 0]
            
        elif action == 1:  # Sap action
            dx = round(offset_x[i])
            dy = round(offset_y[i])
            env_actions[unit] = [5, dx, dy]
    
    return env_actions

# Example usage
if __name__ == "__main__":
    # Example environment actions
    env_actions = action
    
    # Convert to network format
    network_actions = env_to_network_actions(env_actions)

    print("Network format:")
    for k, v in network_actions.items():
        print(f"{k}:\n{v}\n")

NameError: name 'action' is not defined

In [21]:
parsed = parse_observation(obs)

TypeError: list indices must be integers or slices, not str

In [22]:
parsed

NameError: name 'parsed' is not defined

In [23]:
step = data_1["steps"][110]

for i, each_player in enumerate(step):
    obs = each_player["observation"]["obs"]
    obs = json.loads(obs)
    print(f"\nPlayer {i}:")
    print(parse_observation(obs))
    
    total_energy = obs['units']['energy'][i]
    total_energy = sum(num for num in total_energy if num != -1)
    
    
    print(GameStats(
        round_won=obs['team_wins'][i] ,
        relic_points=obs['team_points'][i],
        total_energy=total_energy,
        relic_tiles_found=np.mean(obs['relic_nodes_mask']),
        area_explored=np.mean(obs['sensor_mask'])
    ))


Player 0:
{'map_features': {'energy': tensor([[[ 0.,  2.,  4.,  5.,  6.,  7.,  7.,  7.,  7.,  7., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [ 5.,  7.,  7.,  7.,  6.,  6.,  6.,  6.,  6.,  7., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [ 7.,  5.,  3., -1., -1., -1., -1., -2., -1.,  0., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [ 2., -1., -1., -1., -1., -5., -5., -5., -6., -6., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., 

In [24]:
parse_observation(obs)["map_features"]["energy"]

tensor([[[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1.

In [25]:
class LuxGameParser:
    def __init__(self, map_size=24, max_units=16):
        self.map_size = map_size
        self.max_units = max_units
    
    def parse_game_file(self, file_path: str) -> List[GameStats]:
        with open(file_path, 'r') as f:
            game_data = json.load(f)
        env_stats = [[],[]]
        game_stats = [[],[]]
        for step in game_data['steps']:
            for i, each_player in enumerate(step):
                obs = each_player["observation"]["obs"]
                obs = json.loads(obs)
                env_stats[i].append(parse_observation(obs))
                
                total_energy = obs['units']['energy'][i]
                total_energy = sum(num for num in total_energy if num != -1)
                
                game_stats[i].append(GameStats(
                    round_won=obs['team_wins'][i] ,
                    relic_points=obs['team_points'][i],
                    total_energy=total_energy,
                    relic_tiles_found=np.mean(obs['relic_nodes_mask']),
                    area_explored=np.mean(obs['sensor_mask'])
                ))
                
        return env_stats, game_stats

In [26]:
luxParse = LuxGameParser()


In [27]:
%%time
env_stats, game_stats = luxParse.parse_game_file( glob(file_locs)[2])

CPU times: user 241 ms, sys: 68 ms, total: 309 ms
Wall time: 310 ms


In [28]:
env_stats[0][0].keys()

dict_keys(['map_features', 'sensor_mask', 'units', 'units_mask', 'relic_nodes', 'relic_nodes_mask'])

In [29]:
env_stats[0][0].keys()

dict_keys(['map_features', 'sensor_mask', 'units', 'units_mask', 'relic_nodes', 'relic_nodes_mask'])

In [30]:
import torch
import torch.nn as nn

class SpatialEncoder(nn.Module):
    def __init__(self, width=24, height=24):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),  # 3 channels: energy, tile_type, sensor_mask
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(64 * (width//2) * (height//2), 256)

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x.view(x.size(0), -1))

class UnitProcessor(nn.Module):
    def __init__(self, max_units=10):
        super().__init__()
        self.max_units = max_units
        self.fc1 = nn.Linear(3, 64)  # position (2) + energy (1)
        self.fc2 = nn.Linear(64, 128)
        
    def forward(self, positions, energy, mask):
        # Reshape it to add it in  the last deminsion
        energy = energy.unsqueeze(-1) 
        # Combine position and energy
        features = torch.cat([positions, energy], dim=-1)  # Shape: (batch, teams, units, 3)
        
        # Process each unit
        x = torch.relu(self.fc1(features))
        x = torch.relu(self.fc2(x))
        
        # Apply mask
        x = x * mask.unsqueeze(-1)
        
        # Pool over units dimension
        return torch.max(x, dim=2)[0]  # Shape: (batch, teams, 128)

class RelicProcessor(nn.Module):
    def __init__(self, max_relics=10):
        super().__init__()
        self.fc1 = nn.Linear(2, 64)  # position only
        self.fc2 = nn.Linear(64, 128)
        
    def forward(self, positions, mask):
        x = torch.relu(self.fc1(positions))
        x = torch.relu(self.fc2(x))
        
        # Apply mask
        x = x * mask.unsqueeze(-1)
        
        # Pool over relics
        return torch.max(x, dim=1)[0]

class LuxAIObservationEncoder(nn.Module):
    def __init__(self, width=24, height=24, max_units=10, max_relics=10):
        super().__init__()
        self.spatial_encoder = SpatialEncoder(width, height)
        self.unit_processor = UnitProcessor(max_units)
        self.relic_processor = RelicProcessor(max_relics)

        # Calculate total embedding dimension
        spatial_dim = 256
        units_dim = 128 * 2  # For 2 teams
        relic_dim = 128
        total_dim = spatial_dim + units_dim + relic_dim
        
        self.output_layer = nn.Linear(total_dim, 512)

    def forward(self, obs):
        # Process spatial features
        spatial_features = torch.cat([
            obs['map_features']['energy'].unsqueeze(1),
            obs['map_features']['tile_type'].unsqueeze(1),
            obs['sensor_mask'].unsqueeze(1)
        ], dim=1)
        spatial_embed = self.spatial_encoder(spatial_features)

        # Process units
        unit_embed = self.unit_processor(
            obs['units']['position'],
            obs['units']['energy'],
            obs['units_mask']
        )
        unit_embed = unit_embed.reshape(unit_embed.size(0), -1)  # Flatten teams dimension

        # Process relics
        relic_embed = self.relic_processor(
            obs['relic_nodes'],
            obs['relic_nodes_mask']
        )

        # Combine all embeddings
        combined = torch.cat([
            spatial_embed,
            unit_embed,
            relic_embed
        ], dim=1)

        return self.output_layer(combined)

class GameStateProcessor:
    def process_observation(self, obs):
        """Convert raw observation dict into tensor format"""
        return {
            'map_features': {
                'energy': torch.FloatTensor(obs['obs']['map_features']['energy']),
                'tile_type': torch.FloatTensor(obs['obs']['map_features']['tile_type'])
            },
            'sensor_mask': torch.FloatTensor(obs['obs']['sensor_mask']),
            'units': {
                'position': torch.FloatTensor(obs['obs']['units']['position']),
                'energy': torch.FloatTensor(obs['obs']['units']['energy'])
            },
            'units_mask': torch.FloatTensor(obs['obs']['units_mask']),
            'relic_nodes': torch.FloatTensor(obs['obs']['relic_nodes']),
            'relic_nodes_mask': torch.FloatTensor(obs['obs']['relic_nodes_mask'])
        }

In [31]:
l = LuxAIObservationEncoder()

In [32]:
obs = env_stats[0][0]

In [33]:
spatial_features = torch.cat([
            obs['map_features']['energy'].unsqueeze(1),
            obs['map_features']['tile_type'].unsqueeze(1),
            obs['sensor_mask'].unsqueeze(1)
        ], dim=1)

In [34]:
l(sample).shape

NameError: name 'sample' is not defined

In [35]:
max([j.total_energy for i in game_stats for j in i ])

2606

In [36]:
def calculate_reward(state):
    # Energy transformation function
    def transform_energy(energy):
        x = energy / 3000  # normalize to 0-1
        return (x + 1 - (1-x)**3) / 2
    
    # Core components
    energy_score = transform_energy(state.total_energy)
    
    # Final reward calculation with weights
    reward = (
        1000 * state.round_won +           # Highest priority - winning
        200 * state.relic_points +         # High priority - scoring
        50 * energy_score +                # Medium priority - energy management
        10 * state.area_explored +         # Low priority - exploration
        5 * (state.relic_tiles_found)      # Bonus for finding relic tiles
    )
    
    return reward

In [37]:
game_stats[0][0]

GameStats(round_won=0, relic_points=0, total_energy=0, relic_tiles_found=0.0, area_explored=0.0)

In [38]:
from pathlib import Path

In [39]:
class GameDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir: str, split: str = 'train', val_ratio: float = 0.2, seed: int = 42):
        self.parser = LuxGameParser()
        
        files = glob(data_dir+"/*/*.json") # Sort for reproducibility
        print(files)
        np.random.seed(seed)
        split_idx = int(len(files) * (1 - val_ratio))
        
        # Split files deterministically
        self.files = files[:split_idx] if split == 'train' else files[split_idx:]
        
        # Pre-load all game stats
        # self.games = []
        # for file in self.files:
        #     self.games.extend(self.parser.parse_game_file(str(file)))
    
    def __len__(self):
        return len(self.games)
    
    def __getitem__(self, idx):
        print(self.files[idx])
        env_stats, game_stats_all = luxParse.parse_game_file( self.files[idx])
        print(game_stats_all[0])
        
        labels = torch.tensor([(
            game_stats.round_won,
            game_stats.relic_points,
            game_stats.total_energy,
            game_stats.units_killed,
            game_stats.relic_tiles_found,
            game_stats.area_explored) for game_stats in game_stats_all])
        
        return env_stats, labels

In [40]:
g = GameDataset("../datacrawl/crawled_data")

['../datacrawl/crawled_data/61723086/data.json', '../datacrawl/crawled_data/59230687/data.json', '../datacrawl/crawled_data/60626358/data.json', '../datacrawl/crawled_data/61062532/data.json', '../datacrawl/crawled_data/61445945/data.json', '../datacrawl/crawled_data/61167820/data.json', '../datacrawl/crawled_data/60372112/data.json', '../datacrawl/crawled_data/61532966/data.json', '../datacrawl/crawled_data/61091037/data.json', '../datacrawl/crawled_data/61790536/data.json', '../datacrawl/crawled_data/61822992/data.json', '../datacrawl/crawled_data/60904849/data.json', '../datacrawl/crawled_data/61212581/data.json', '../datacrawl/crawled_data/60565567/data.json', '../datacrawl/crawled_data/61342722/data.json', '../datacrawl/crawled_data/61820795/data.json', '../datacrawl/crawled_data/60642122/data.json', '../datacrawl/crawled_data/61764224/data.json', '../datacrawl/crawled_data/61770794/data.json', '../datacrawl/crawled_data/59194601/data.json', '../datacrawl/crawled_data/60514228/dat

In [41]:
g[0]

../datacrawl/crawled_data/61723086/data.json
[GameStats(round_won=0, relic_points=0, total_energy=0, relic_tiles_found=0.0, area_explored=0.0), GameStats(round_won=0, relic_points=0, total_energy=100, relic_tiles_found=0.0, area_explored=0.015625), GameStats(round_won=0, relic_points=0, total_energy=98, relic_tiles_found=0.0, area_explored=0.020833333333333332), GameStats(round_won=0, relic_points=0, total_energy=96, relic_tiles_found=0.0, area_explored=0.027777777777777776), GameStats(round_won=0, relic_points=0, total_energy=195, relic_tiles_found=0.0, area_explored=0.034722222222222224), GameStats(round_won=0, relic_points=0, total_energy=192, relic_tiles_found=0.0, area_explored=0.041666666666666664), GameStats(round_won=0, relic_points=0, total_energy=189, relic_tiles_found=0.0, area_explored=0.05381944444444445), GameStats(round_won=0, relic_points=0, total_energy=286, relic_tiles_found=0.0, area_explored=0.06597222222222222), GameStats(round_won=0, relic_points=0, total_energy=2

AttributeError: 'list' object has no attribute 'round_won'

In [42]:
glob(file_locs)[2]

'../datacrawl/crawled_data/60626358/data.json'

In [43]:
glob(file_locs)[0]

'../datacrawl/crawled_data/61723086/data.json'

In [44]:
game_stats[0][-2]

GameStats(round_won=3, relic_points=588, total_energy=1718, relic_tiles_found=0.6666666666666666, area_explored=0.4409722222222222)

In [45]:
obs['units']['energy'][i]

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [46]:
total_energy

349

In [47]:
step = data_1["steps"][102]


for i, each_player in enumerate(step):
    observation = each_player["observation"]
    obs = observation["obs"]
    obs = json.loads(obs)
    print(f"\nPlayer {i}:")
    print(parse_observation(obs))
    print(GameStats(
        match_won=obs['team_wins'][i] ,
        round_won=observation["reward"],  # Compare with other player
        relic_points=obs['team_points'][i],
        total_energy=np.sum(obs['units']['energy'][i]),
        relic_tiles_found=np.mean(obs['relic_nodes_mask']),
        area_explored=np.mean(obs['sensor_mask'])
    ))


Player 0:
{'map_features': {'energy': tensor([[[ 0.,  2.,  4., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [ 5.,  7.,  7., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [ 7.,  5.,  3., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., 

TypeError: GameStats.__init__() got an unexpected keyword argument 'match_won'

In [48]:
obs

{'units': {'position': [[[0, 0],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1]],
   [[-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1]]],
  'energy': [[100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
   [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]},
 'units_mask': [[True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False]],
 'sensor_mask': [[True,
   True,
   True,
 

In [49]:
step = data_1["steps"][101]

for each_player in step:
    obs = each_player["observation"]["obs"]
    obs = json.loads(obs)
    print(parse_observation(obs))
    print(GameStats(
            match_won=obs['team_wins'][0] > 0,  # Assuming player 0
            round_won=obs['team_points'][0] > obs['team_points'][1],
            relic_points=obs['team_points'][0],
            total_energy=np.sum(obs['units']['energy'][0]),
            relic_tiles_found=np.mean(obs['relic_nodes_mask']),
            area_explored=np.mean(obs['sensor_mask'])
        ))

{'map_features': {'energy': tensor([[[-1., -1., -1., -1., -1.,  1.,  1.,  0.,  0., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.,  1.,  1.,  0.,  0., -1., -1., -2., -3., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1.,  4.,  5.,  5.,  6.,  6.,  5.,
           2., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  3.,  4.,  5.,  6.,  8.,
           8., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1.,  0.,  1.,  4.,
           7., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
         [-1., -1., -1.,  1.,  2.,  2.,  1.,  1., -1., -1., -2., -4., -4., -3.,
           1.,  6., -1., -1., -1., -1., -1., -

TypeError: GameStats.__init__() got an unexpected keyword argument 'match_won'

In [50]:
each_player.keys()

dict_keys(['action', 'info', 'observation', 'reward', 'status'])

In [51]:
data_1["steps"][0][0]["observation"].keys()

dict_keys(['obs', 'player', 'remainingOverageTime', 'reward', 'step'])

In [52]:
obs = json.loads(obs)

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [53]:
data_1["steps"][1][1]

{'action': [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 'info': {},
 'observation': {'obs': '{"units": {"position": [[[-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1]], [[23, 23], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1]]], "energy": [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]}, "units_mask": [[false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false], [true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false]], "sens

In [54]:
obs

{'units': {'position': [[[11, 8],
    [0, 7],
    [1, 10],
    [9, 7],
    [13, 2],
    [5, 11],
    [1, 10],
    [6, 12],
    [8, 5],
    [9, 8],
    [8, 13],
    [11, 7],
    [12, 7],
    [12, 9],
    [4, 12],
    [9, 11]],
   [[-1, -1],
    [-1, -1],
    [-1, -1],
    [14, 11],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1]]],
  'energy': [[0, 0, 0, 0, 20, 0, 0, 69, 0, 0, 41, 0, 77, 7, 94, 117],
   [-1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]},
 'units_mask': [[True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False]],
 'sensor_mask': [[False,
   False,
   False,
   False,
   False,
   True,
   True,
   Tr

In [55]:
steps[1][0].keys()

NameError: name 'steps' is not defined

In [56]:
replay = steps[10][0]["info"]["replay"]

NameError: name 'steps' is not defined

In [57]:
steps

NameError: name 'steps' is not defined

In [58]:
for i in replay["observations"][0].keys():
    print(i)

NameError: name 'replay' is not defined

In [59]:
observations = replay["observations"][0]

NameError: name 'replay' is not defined

In [60]:
observations["energy_nodes"]

NameError: name 'observations' is not defined

In [61]:
observations["map_features"].keys()

NameError: name 'observations' is not defined

In [62]:
energy = observations["map_features"]["energy"]
tile_type = observations["map_features"]["tile_type"]

NameError: name 'observations' is not defined

In [63]:
observations["units"].keys()

NameError: name 'observations' is not defined

In [64]:
import torch
torch.tensor(observations["vision_power_map"]).shape

NameError: name 'observations' is not defined

In [65]:
vision_power_map = observations["vision_power_map"][1]

NameError: name 'observations' is not defined

In [66]:
observations["units_mask"]

NameError: name 'observations' is not defined

In [67]:
sample_input

NameError: name 'sample_input' is not defined

In [68]:
for i in sample_input["obs"].keys():
    print(i)#["sensor_mask"]

NameError: name 'sample_input' is not defined

In [69]:
steps[0]["info"].keys()

NameError: name 'steps' is not defined

In [70]:
steps[1].keys()

NameError: name 'steps' is not defined

In [ ]:
len(steps[0])#["info"]["replay"].keys()

2

In [53]:
import numpy as np
energy = steps[0]["info"]["replay"]["observations"][0]["units"]["energy"] 
energy = np.array(energy)
energy.shape

(2, 16, 1)

In [71]:
len(energy)

NameError: name 'energy' is not defined

In [72]:
data_1["rewards"]

[3, 2]

In [73]:
import torch 
def process_game_json(self, game_data):
    """
    Process a complete game JSON into observation tensors and metrics
    Returns:
        - team_0_obs: Tensor of observations for team 0
        - team_1_obs: Tensor of observations for team 1
        - metrics: Tensor of performance metrics
    """
    match_observations = []
    for step_data in game_data['steps']:
        if len(step_data) < 2:  # Sometimes steps can be empty
            continue
            
        obs_0 = self._process_observation(step_data[0]['observation'], step_data[0]['info'])
        obs_1 = self._process_observation(step_data[1]['observation'], step_data[1]['info'])
        match_observations.append((obs_0, obs_1))
    
    # Stack all observations into a single tensor per team
    team_0_obs = torch.stack([obs[0] for obs in match_observations])
    team_1_obs = torch.stack([obs[1] for obs in match_observations])
    
    # Calculate metrics
    metrics = self._calculate_metrics(game_data)
    return 

In [74]:
game_data = data_1#["steps"][0]

In [75]:
# game_data#[0]["replay"]

In [76]:
def _process_observation(info_data,obs_data):
    """Convert a single observation into tensor format"""
    replay_data = info_data['replay']['observations'][0]
    
    # Process map features
    map_features = {
        'energy': torch.tensor(replay_data['map_features']['energy'], dtype=torch.float32),
        'tile_type': torch.tensor(replay_data['map_features']['tile_type'], dtype=torch.float32)
    }
    
    # Process vision power map
    vision_power = torch.tensor(replay_data['vision_power_map'], dtype=torch.float32)
    
    # Process units mask
    units_mask = torch.tensor(replay_data['units_mask'], dtype=torch.float32)
    
    # Process units
    units = {
        'position': torch.tensor(replay_data['units']['position'], dtype=torch.float32),
        'energy': torch.tensor(replay_data['units']['energy'], dtype=torch.float32)
    }
    
    # Process nodes
    energy_nodes = torch.tensor(replay_data['energy_nodes'], dtype=torch.float32)
    relic_nodes = torch.tensor(replay_data['relic_nodes'], dtype=torch.float32)
    relic_configs = torch.tensor(replay_data['relic_node_configs'], dtype=torch.float32)
    
    return {
        'map_features': map_features,
        'vision_power_map': vision_power,
        'units_mask': units_mask,
        'units': units,
        'energy_nodes': energy_nodes,
        'relic_nodes': relic_nodes,
        'relic_node_configs': relic_configs
    }

In [77]:
json.loads(step_data[1]["observation"]["obs"])

NameError: name 'step_data' is not defined

In [86]:
match_observations = []
for step_data in game_data['steps']:
    if len(step_data) < 2:  # Sometimes steps can be empty
        continue
        
    obs_0 = _process_observation(step_data[0]['observation'], step_data[0]['info'])
    obs_1 = _process_observation(step_data[1]['observation'], step_data[1]['info'])
    match_observations.append((obs_0, obs_1))

# Stack all observations into a single tensor per team
team_0_obs = torch.stack([obs[0] for obs in match_observations])
team_1_obs = torch.stack([obs[1] for obs in match_observations])

# Calculate metrics
metrics = self._ca

KeyError: 'replay'